In [1]:
%run "..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

31056


In [15]:
# Required plotting setting
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
plt.rc('font', family='serif')
plt.rc('font', serif='Arial')
_font_size = 7.5

In [2]:
#figure_folder = r'L:\Shiwei\RNA_MERFISH_analysis\Merged_nonclear' # from shiwei
figure_folder = r'\\10.245.74.158\Chromatin_NAS_8\Analyzed_data\MouseBrain_PostAnalysis\Figures_0817'
print(figure_folder)

figure_folder = r'\\10.245.74.158\Chromatin_NAS_8\Analyzed_data\MouseBrain_PostAnalysis\Figures_0817'
comp_figure_folder = os.path.join(figure_folder, 'AB_compartment')

if not os.path.exists(comp_figure_folder):
    print(f"Create: {comp_figure_folder}")
    os.makedirs(comp_figure_folder)
else:
    print(f"Use: {comp_figure_folder}")
    

\\10.245.74.158\Chromatin_NAS_8\Analyzed_data\MouseBrain_PostAnalysis\Figures_0817
Use: \\10.245.74.158\Chromatin_NAS_8\Analyzed_data\MouseBrain_PostAnalysis\Figures_0817\AB_compartment


In [3]:
# load contact matrix
postanalysis_folder = r'\\10.245.74.158\Chromatin_NAS_8\Analyzed_data\MouseBrain_PostAnalysis'
print(postanalysis_folder)

ensemble_contact_filename = os.path.join(postanalysis_folder, 'ensemble_contactDict_600.pkl')
print(ensemble_contact_filename)
ensemble_contactDict = np.load(ensemble_contact_filename, allow_pickle=True)

\\10.245.74.158\Chromatin_NAS_8\Analyzed_data\MouseBrain_PostAnalysis
\\10.245.74.158\Chromatin_NAS_8\Analyzed_data\MouseBrain_PostAnalysis\ensemble_contactDict_600.pkl


## load old version of codebook

In [4]:
import pandas as pd
merged_data_folder = r'\\crick\SSD_0\Shiwei\DNA_MERFISH_analysis\20220402plus0316_MERFISH'
print(merged_data_folder)
codebook = pd.read_csv(os.path.join(merged_data_folder, 'merged_codebook_w_celltype.csv'))
bit_columns = [_n for _n in codebook.columns if 'NDB' in _n]

\\crick\SSD_0\Shiwei\DNA_MERFISH_analysis\20220402plus0316_MERFISH


In [5]:
from ImageAnalysis3.structure_tools import distance
ensemble_contact_mat, _chr_edges, _chr_names = distance.assemble_ChrDistDict_2_Matrix(
    ensemble_contactDict, codebook, use_cis=False, use_trans=False, sort_by_region=False)

In [6]:
subclass_2_normedPC1_filename = os.path.join(postanalysis_folder, 'subclass_2_normedPC1.pkl')
subclass_2_normedPC1 = pickle.load(open(subclass_2_normedPC1_filename, 'rb'))

In [7]:
subclass_2_normedPC1['ensemble'].shape

(1955,)

In [8]:
len(codebook)

1956

In [9]:
from ImageAnalysis3.figure_tools import _single_col_width, _font_size, _ticklabel_size,_ticklabel_width

In [10]:
# define AB
ensemble_AB_dict = {}
AB_th = -0.5
ensemble_AB_dict['A'] = np.where(subclass_2_normedPC1['ensemble'] > AB_th)[0]
ensemble_AB_dict['B'] = np.where(subclass_2_normedPC1['ensemble'] <= AB_th)[0]

In [22]:
%matplotlib inline
from matplotlib import cm
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable

#plt.style.use('dark_background')
plt.style.use('default')
fig, ax1 = plt.subplots(figsize=(_single_col_width,_single_col_width), dpi=200)

# create a color map
current_cmap = cm.get_cmap('seismic').copy()
current_cmap.set_bad(color=[0.5,0.5,0.5,1])

vmin,vmax = 1e-3, 1e-2

_pf = ax1.imshow(ensemble_contact_mat, 
                 cmap=current_cmap, norm=LogNorm(vmin=vmin, vmax=vmax, ))

ax1.set_yticks((_chr_edges[1:] + _chr_edges[:-1])/2,)
ax1.set_yticklabels(_chr_names,)

#                                              ticks=
#                                              tick_labels=_chr_names,

ax1.hlines(_chr_edges-0.5, 0, len(ensemble_contact_mat), color='yellow', linewidth=0.2)
ax1.vlines(_chr_edges-0.5, 0, len(ensemble_contact_mat), color='yellow', linewidth=0.2)
ax1.xaxis.set_tick_params(which='both', labelbottom=True)
ax1.yaxis.set_tick_params(which='both', labelleft=True)
ax1.set_xlim([0, len(ensemble_contact_mat)])
ax1.set_ylim([len(ensemble_contact_mat), 0])

ax1.tick_params('both', labelsize=_font_size-1, 
                width=_ticklabel_width, length=_ticklabel_size,
                pad=1, labelleft=True, labelbottom=False) # remove bottom ticklabels for ax1
[i[1].set_linewidth(_ticklabel_width) for i in ax1.spines.items()]

# locate ax1
divider = make_axes_locatable(ax1)
# colorbar ax
cax = divider.append_axes('right', size='6%', pad="4%")
cbar = plt.colorbar(_pf,cax=cax, ax=ax1, ticks=[vmin,vmax])
cbar.ax.tick_params('both', labelsize=_font_size, 
                width=_ticklabel_width, length=_ticklabel_size-1,
                pad=1, labelleft=False) # remove bottom ticklabels for ax1
cbar.outline.set_linewidth(_ticklabel_width)
cbar.set_label('Pearson correlation', 
               fontsize=_font_size, labelpad=0, rotation=270)

# create bottom ax
bot_ax = divider.append_axes('bottom', size='10%', pad="0%", 
                             sharex=ax1, xticks=[])
bot_ax.bar(ensemble_AB_dict['A'], subclass_2_normedPC1['ensemble'][ensemble_AB_dict['A']]-AB_th, 
           color='r',width=1, bottom=0)
bot_ax.bar(ensemble_AB_dict['B'], subclass_2_normedPC1['ensemble'][ensemble_AB_dict['B']]-AB_th, 
           color='b',width=1, bottom=0)

bot_ax.set_xticks((_chr_edges[1:] + _chr_edges[:-1])/2,)
bot_ax.set_xticklabels(_chr_names, rotation=60)
bot_ax.tick_params('x', labelsize=_font_size-1, 
                width=_ticklabel_width, length=_ticklabel_size,
                pad=1, labelleft=False, labelbottom=True) # remove bottom ticklabels for ax1
bot_ax.tick_params('y', labelsize=_font_size-1, 
                width=_ticklabel_width, length=0,
                pad=1, labelleft=False, labelbottom=True) # remove bottom ticklabels for ax1
bot_ax.set_ylabel('PC1', fontsize=_font_size-1, labelpad=1)
#bot_ax.set_ylim([-10,10])
#bot_ax.set_yticks([0.5,-0.5])
#bot_ax.set_yticklabels(["A","B"])
#plt.savefig(os.path.join(comp_figure_folder, f"trans_ensemble_contact_AB.pdf"),
#            transparent=True, bbox_inches='tight', pad_inches=0)

Text(0, 0.5, 'PC1')

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x0000024F233681F0> (for post_execute):


RuntimeError: In FT2Font: Can not load face (error code 0x55)

RuntimeError: In FT2Font: Can not load face (error code 0x55)

<Figure size 450x450 with 3 Axes>

In [12]:
os.path.join(comp_figure_folder, f"trans_ensemble_contact_AB.pdf")

'\\\\10.245.74.158\\Chromatin_NAS_8\\Analyzed_data\\MouseBrain_PostAnalysis\\Figures_0817\\AB_compartment\\trans_ensemble_contact_AB.pdf'

In [18]:
print(matplotlib.__file__)

E:\Users\puzheng\AppData\anaconda3\envs\ia3_postanalysis\lib\site-packages\matplotlib\__init__.py
